Preprocessing file for data cleaning and transformation before analysis.
This notebook includes code for:
1. Data loading and cleaning
2. Graph Construction
3. Complex Networks Measures Computation
4. Adding Complex Networks Features to the dataframe

Importing Libraries

In [1]:
import json
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

# datasets is a list of available datasets descriptions containing: path, key columns names, and suitable complex network features
from src.data.dataset_info import datasets

## 1. Data loading and cleaning

In [2]:
dataset = datasets[0]
name = dataset.name
print("dataset: {}".format(name))

path = "./datasets/original/{}.pkl".format(name)
new_path = "./datasets/preprocessed/{}.pkl".format(name)
graph_path = "./datasets/preprocessed/graph_{}.gexf".format(name)
df = pd.read_pickle(path)

dataset: cic_ton_iot


### Dropping infinity values, Nan values, and duplicates

In [3]:
# converting all infinity values into nan then dropping all records containing nan values
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(axis=0, how='any', inplace=True)

df.drop_duplicates(subset=list(set(df.columns) - set([dataset.timestamp_col, dataset.flow_id_col])), keep="first", inplace=True)

### Dataset Properties

calculating main dataset properties and saving them in a json file

In [4]:
total_count = len(df)

properties = {
    "name": dataset.name,
    "length": total_count,
}

num_benign = len(df[df['Label'] == 0])
num_attack = len(df[df['Label'] == 1])

properties["num_benign"] = num_benign
properties["percentage_of_benign_records"] = ((num_benign * 100)/total_count)

properties["num_attack"] = num_attack
properties["percentage_of_attack_records"] = ((num_attack * 100)/total_count)

properties["attacks"] = list(df["Attack"].unique())  # .to_list()

filename = ('./datasets_properties/{}.json'.format(dataset.name))
outfile = open(filename, 'w')
outfile.writelines(json.dumps(properties))
outfile.close()

## 2. Graph Construction

Graph construction from the records in the dataset.<br>
Nodes are specified by IP addresses. <br>
If there exists atleast one network flow between two different IP addresses, an edge will be created. <br>
Another way can be considered is to use MultiDiGraph class. However, some centralities will not work in addition to transitivity.

In [5]:
G = nx.from_pandas_edgelist(
        df,
        source=dataset.src_ip_col,
        target=dataset.dst_ip_col,
        create_using=nx.DiGraph()
    )

In [6]:
G.remove_nodes_from(list(nx.isolates(G)))

Specifying the communities in the graph using the methods get_communities. <br>
Since communities can be calculated using different methods, and we want to use get communites at different stages of the code, we implemented it in a separate file, so a change will be done one time.

In [7]:
import igraph as ig
G1 = ig.Graph.from_networkx(G)
part = G1.community_infomap()

communities = []
for com in part:
    communities.append([G1.vs[node_index]['_nx_name'] for node_index in com])

print(f"==>> number of communities: {len(communities)}")
for com in communities:
    print(f"==>> com: {len(com)}")

==>> number of communities: 100
==>> com: 69
==>> com: 16
==>> com: 390
==>> com: 130242
==>> com: 27
==>> com: 5
==>> com: 1040
==>> com: 60
==>> com: 127
==>> com: 6
==>> com: 117
==>> com: 2
==>> com: 59
==>> com: 70
==>> com: 2
==>> com: 2
==>> com: 142
==>> com: 2
==>> com: 171
==>> com: 4160
==>> com: 2
==>> com: 3
==>> com: 2
==>> com: 2
==>> com: 9
==>> com: 2
==>> com: 2
==>> com: 2
==>> com: 108
==>> com: 218
==>> com: 390
==>> com: 2
==>> com: 1014
==>> com: 26
==>> com: 2
==>> com: 2
==>> com: 2
==>> com: 2
==>> com: 2
==>> com: 49
==>> com: 233
==>> com: 4
==>> com: 2
==>> com: 57
==>> com: 6
==>> com: 2
==>> com: 3
==>> com: 2
==>> com: 66
==>> com: 2
==>> com: 6
==>> com: 2
==>> com: 469
==>> com: 2
==>> com: 2
==>> com: 2
==>> com: 382
==>> com: 326
==>> com: 2
==>> com: 11
==>> com: 60
==>> com: 71
==>> com: 3
==>> com: 2
==>> com: 2
==>> com: 4
==>> com: 2
==>> com: 1908
==>> com: 69
==>> com: 36
==>> com: 123
==>> com: 26
==>> com: 149
==>> com: 2
==>> com: 6
==>> co

## 3. Complex Networks Measures Computation

### 3.1. Computing Graph-level Measures

In [8]:
properties = {}

properties["number_of_nodes"] = G.number_of_nodes()
properties["number_of_edges"] = G.number_of_edges()

degrees = [degree for _, degree in G.degree()]
properties["max_degree"] = max(degrees)
properties["avg_degree"] = sum(degrees) / len(degrees)

In [9]:
properties["transitivity"] = nx.transitivity(G)

In [10]:
properties["density"] =  nx.density(G)

In [11]:
# Assuming G is your graph and communities is a list of sets, where each set contains the nodes in a community

# Step 1: Map each node to its community
node_to_community = {}
for community_index, community in enumerate(communities):
    for node in community:
        node_to_community[node] = community_index

# Step 2: Count inter-cluster edges efficiently
inter_cluster_edges = 0
for u, v in G.edges():
    # Directly check if u and v belong to different communities
    if node_to_community[u] != node_to_community[v]:
        inter_cluster_edges += 1


properties["mixing_parameter"] = inter_cluster_edges / G.number_of_edges()

In [12]:
properties["modularity"] = nx.community.modularity(G, communities)

In [13]:
filename = ('./datasets_properties/{}.json'.format("graph_" + name))
outfile = open(filename, 'w')
outfile.writelines(json.dumps(properties))
outfile.close()

properties

{'number_of_nodes': 142669,
 'number_of_edges': 352453,
 'max_degree': 58036,
 'avg_degree': 4.940849098262412,
 'transitivity': 0.144479693894849,
 'density': 1.7315898092993565e-05,
 'mixing_parameter': 0.002164827650778969,
 'modularity': 0.07225186954453378}

Using the graph-level metrics, suitable complex networks should be specified and added to the corresponding dataset in the list in src.data.dataset_info

### 3.2. Computing Node-level Measures

In [14]:
community_labels = {}
for i, community in enumerate(communities):
    for node in community:
        community_labels[node] = i

nx.set_node_attributes(G, community_labels, "new_community")

In [15]:
# getting inter and itra graph, to calculate the local and global variations of each centrality
from src.network.network_features import separate_graph

intra_graph, inter_graph = separate_graph(G, communities)

In [16]:
from src.network.network_features import cal_betweenness_centrality

if "betweenness" in dataset.cn_measures:
    nx.set_node_attributes(G, cal_betweenness_centrality(G), "betweenness")
    print("calculated")

calculated


In [17]:
if "local_betweenness" in dataset.cn_measures:
    nx.set_node_attributes(G, cal_betweenness_centrality(intra_graph), "local_betweenness")
    print("calculated")

calculated


In [18]:
if "global_betweenness" in dataset.cn_measures:
    nx.set_node_attributes(G, cal_betweenness_centrality(inter_graph), "global_betweenness")
    print("calculated")

In [19]:
if "degree" in dataset.cn_measures:
    nx.set_node_attributes(G, nx.degree_centrality(G), "degree")
    print("calculated")

calculated


In [20]:
if "local_degree" in dataset.cn_measures:
    nx.set_node_attributes(G, nx.degree_centrality(intra_graph), "local_degree")
    print("calculated")

calculated


In [21]:
if "global_degree" in dataset.cn_measures:
    nx.set_node_attributes(G, nx.degree_centrality(inter_graph), "global_degree")
    print("calculated")

In [22]:
if "eigenvector" in dataset.cn_measures:
    nx.set_node_attributes(G, nx.eigenvector_centrality(G, max_iter=600), "eigenvector")
    print("calculated")


calculated


In [23]:
if "local_eigenvector" in dataset.cn_measures:
    nx.set_node_attributes(G, nx.eigenvector_centrality(intra_graph), "local_eigenvector")
    print("calculated")

In [24]:
if "global_eigenvector" in dataset.cn_measures:
    nx.set_node_attributes(G, nx.eigenvector_centrality(inter_graph), "global_eigenvector")
    print("calculated")

In [25]:
if "closeness" in dataset.cn_measures:
    nx.set_node_attributes(G, nx.closeness_centrality(G), "closeness")
    print("calculated")

calculated


In [26]:
if "local_closeness" in dataset.cn_measures:
    nx.set_node_attributes(G, nx.closeness_centrality(intra_graph), "local_closeness")
    print("calculated")

In [27]:
if "global_closeness" in dataset.cn_measures:
    nx.set_node_attributes(G, nx.closeness_centrality(inter_graph), "global_closeness")
    print("calculated")

In [29]:
if "pagerank" in dataset.cn_measures:
    nx.set_node_attributes(G, nx.pagerank(G, alpha=0.85), "pagerank")
    print("calculated")

calculated


In [31]:
if "local_pagerank" in dataset.cn_measures:
    nx.set_node_attributes(G, nx.pagerank(intra_graph, alpha=0.85), "local_pagerank")
    print("calculated")

calculated


In [32]:
if "global_pagerank" in dataset.cn_measures:
    nx.set_node_attributes(G, nx.pagerank(inter_graph, alpha=0.85), "global_pagerank")
    print("calculated")

In [33]:
from src.network.network_features import cal_k_core

if "k_core" in dataset.cn_measures:
    nx.set_node_attributes(G, cal_k_core(G), "k_core")
    print("calculated")

calculated


In [34]:
from src.network.network_features import cal_k_truss
if "k_truss" in dataset.cn_measures:
    nx.set_node_attributes(G, cal_k_truss(G), "k_truss")
    print("calculated")

calculated


In [35]:
from src.network.CommCentralityCode import comm_centreality

if "Comm" in dataset.cn_measures:
    nx.set_node_attributes(G, comm_centreality(G, community_labels), "Comm")
    print("calculated")

calculated


In [36]:
from src.network.modularity_vitality import modularity_vitality

if "mv" in dataset.cn_measures:
    nx.set_node_attributes(G, modularity_vitality(G, part), "mv")
    print("calculated")

In [37]:
nx.write_gexf(G, graph_path)

## 4. Adding Complex Networks Features to the dataframe

In [38]:
features_dicts = {}
for measure in dataset.cn_measures:
    features_dicts[measure] = nx.get_node_attributes(G, measure)
    print(f"==>> features_dicts: {measure , len(features_dicts[measure])}")
    
for feature in dataset.network_features:
        if feature[:3] == "src":
            df[feature] = df.apply(
                lambda row: features_dicts[feature[4:]].get(row[dataset.src_ip_col], -1), axis=1)
        if feature[:3] == "dst":
            df[feature] = df.apply(
                lambda row: features_dicts[feature[4:]].get(row[dataset.dst_ip_col], -1), axis=1)

==>> features_dicts: ('betweenness', 142669)
==>> features_dicts: ('local_betweenness', 142669)
==>> features_dicts: ('degree', 142669)
==>> features_dicts: ('local_degree', 142669)
==>> features_dicts: ('eigenvector', 142669)
==>> features_dicts: ('closeness', 142669)
==>> features_dicts: ('pagerank', 142669)
==>> features_dicts: ('local_pagerank', 142669)
==>> features_dicts: ('k_core', 142669)
==>> features_dicts: ('k_truss', 142669)
==>> features_dicts: ('Comm', 142669)


In [39]:
df.head()

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,...,src_pagerank,dst_pagerank,src_local_pagerank,dst_local_pagerank,src_k_core,dst_k_core,src_k_truss,dst_k_truss,src_Comm,dst_Comm
0,177.30.87.144-192.168.1.1-0-0-0,177.30.87.144,0.0,192.168.1.1,0.0,0.0,25/04/2019 05:18:52 pm,47814343.0,5.0,0.0,...,0.000002,0.000021,0.000002,0.000018,0.090909,1.000000,0.000210,0.001051,0.001926,0.231202
1,167.49.176.28-50.165.192.168-0-0-0,167.49.176.28,0.0,50.165.192.168,0.0,0.0,25/04/2019 05:18:49 pm,2033142.0,2.0,0.0,...,0.000002,0.000029,0.000004,0.000055,0.090909,0.181818,0.000210,0.000315,0.030364,0.937725
2,230.158.52.59-177.21.192.168-0-0-0,230.158.52.59,0.0,177.21.192.168,0.0,0.0,25/04/2019 05:18:37 pm,82877133.0,14.0,0.0,...,0.000002,0.000709,0.000004,0.001335,0.090909,0.363636,0.000210,0.000526,0.028652,0.997676
3,183.68.192.168-1.1.192.168-0-0-0,183.68.192.168,0.0,1.1.192.168,0.0,0.0,25/04/2019 05:18:42 pm,24359.0,2.0,0.0,...,0.000002,0.061140,0.000003,0.007412,0.363636,0.818182,0.000420,0.000631,0.010413,0.031110
4,183.41.192.168-1.1.192.168-0-0-0,183.41.192.168,0.0,1.1.192.168,0.0,0.0,25/04/2019 05:18:42 pm,10239351.0,3.0,0.0,...,0.000002,0.061140,0.000005,0.007412,0.636364,0.818182,0.000526,0.000631,0.018223,0.031110


In [41]:
pd.to_pickle(df, new_path)